<a href="https://colab.research.google.com/github/demekeendalie/sexually_explicit/blob/main/sexually_explicit_comment_classification_using_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# reading the two files from MyDrive of the mounted drive
harashment=pd.read_excel('/content/drive/MyDrive/harashment.xlsx')# sexually expilict Amharic comments
normal=pd.read_excel('/content/drive/MyDrive/Normal comments.xlsx')# normal Amharic comments

In [ ]:
#taking the two frames as one dataframe data(appending)
data = pd.concat([harashment, normal], ignore_index=True)
#data=data[['comments','label']]
data=data.sample(frac=1).reset_index(drop=True)
print(data.tail())

                                                comments  label
34707                      አለን ህዝበ ክርስቲያኑ ከፍተኛ ጥንቃቄ ያድርግ      0
34708               እኔ ማሰብ ማቆም የማልችለው የመጨረሻው ቅዠት አንተ ነህ.      1
34709               ምርጥ ኢትዮጵያዊ እግዚአብሄር እዳንቺ ያሉ ሰዎች ያብዛልን      0
34710  በዱር በገደሉ ከጠላት ጋር ለምትዋደቁ ጀግኖቻችን፣ በትህነግ አፈና ውስጥ ...      0
34711      የአንበጣ መንጋን ለመበተን ከሚያስችሉ ዘዴዎች መካከል የሚከተሉት ይገኛሉ      0


In [ ]:
# pre-processing of the dataset
#normalizarion
import re
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

In [ ]:
#normalization of Amharic characters having the same sound and meaning
data['comments'] = data['comments'].astype(str)
data['comments']=data['comments'].apply(lambda x: normalization(x))

In [ ]:
#pre-processing function
def remove_non_amharic(text):
  # Define a regex pattern for Amharic characters (Unicode range)
    amharic_pattern = r'[^\u1200-\u137F\s]'  # Unicode range for Amharic characters
    amharic_pattern=r'[\u1200-\u137F]*[0-9]+[\u1200-\u137F]*'#removing digits
    amharic_pattern = r'[፡-፨]'  # Unicode range for Amharic punctuation marks
    amharic_pattern=r'[^\w\s]'# removing white spaces
    # Use re.sub to replace non-Amharic characters with an empty string
    cleaned_text = re.sub(amharic_pattern, '', text)
    return cleaned_text

In [ ]:
# processing the data such as removing non-Amharic characters, digit,punctuation marks
data['comments']=data['comments'].apply(remove_non_amharic)
data.tail(20)

,comments,label
34692,ፍትህ ፍትህ ፍትህ ፍትህ ለወላይታ በዝህ የሰብአዊ ጥቃት እና ማንነትን መ...,0
34693,በመጀመሪያ እንኳን ሰላም ተከሰትክ,0
34694,ከህንፃችን አጠገብ ባለ ኮሪደር ላይ እያለፍኩ ነበር እና አንድ ሰው አለፍ...,1
34695,የኮሌጅ ተማሪዎች በፌስቲቫሉ ላይ ልጃገረዶች ላይ እያሳደዱ እና አስተያየት...,1
34696,ከጎረምሳ ጓደኞዬ ጋር የወደዳት ልጅ ግቢ ውስጥ የቆርቆሮ አጥር ዘልዬ የገ...,0
34697,አንተ ደደብ ነህ,1
34698,Ethiopi የሰብአዊ ድርጅቶች በትግራይ ክልል ያለገደብ እንዲንቀሳቀሱ ...,0
34699,እስክንድር ነጋ ባልደራስ ለእውነተኛ ዴሞክራሲ ቀደም ብሎ አምስት ሺህ ብር...,0
34700,በብሎክ ወገብ ዛላህን በማለቴ ደስታዬ ወደር የለውምበዳንኪራ እየጨፈርኩ ሳ...,0
34701,አብቹ,0


In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(data['comments'],data['label'], test_size=0.20, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
# Create a pipeline with TfidfVectorizer and MLPClassifier
model = make_pipeline(TfidfVectorizer(), MLPClassifier(hidden_layer_sizes=(50,), max_iter=50))

# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=(50,), max_iter=50))])

In [ ]:
# Make predictions
from sklearn.metrics import confusion_matrix
# Now you can make predictions
predictions = model.predict(X_test)
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
print(cm)

[[3278  190]
 [ 263 3212]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3468
           1       0.94      0.92      0.93      3475

    accuracy                           0.93      6943
   macro avg       0.93      0.93      0.93      6943
weighted avg       0.93      0.93      0.93      6943



In [ ]:
# prenting each values of the metriics used
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
print(accuracy_score(y_test, predictions))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
print(f1_score(y_test, predictions))

0.9347544289212156
0.9441504997060552
0.9243165467625899
0.934128253598953
